## SQL DW as data source for Azure Databricks

https://docs.databricks.com/spark/latest/data-sources/azure/sql-data-warehouse.html

In [2]:
%scala
Class.forName("com.databricks.spark.sqldw.DefaultSource")

In [3]:
spark.conf.set(
  "fs.azure.account.key.<blobAccountName>.blob.core.windows.net",
  "<Key1/2>")

In [4]:
%sql
CREATE TABLE dbtblFactInternetSales
USING com.databricks.spark.sqldw
OPTIONS (
  url 'jdbc:sqlserver://<dbserver>.database.windows.net:1433;database=<dbName>;user=<USERID>@azlabf5;password=<PASSWORD>;encrypt=true;trustServerCertificate=false;hostNameInCertificate=*.database.windows.net;loginTimeout=30;',
  forwardSparkAzureStorageCredentials 'true',
  dbTable 'FactInternetSales',
  tempDir 'wasbs://<CONTAINERNAME>@<BLOBNAME>.blob.core.windows.net/dbtblFactInternetSales'
);

In [5]:
%sql
CREATE TABLE dbtblDimDate
USING com.databricks.spark.sqldw
OPTIONS (
  url 'jdbc:sqlserver://<dbserver>.database.windows.net:1433;database=<dbName>;user=<USERID>@azlabf5;password=<PASSWORD>;encrypt=true;trustServerCertificate=false;hostNameInCertificate=*.database.windows.net;loginTimeout=30;',
  forwardSparkAzureStorageCredentials 'true',
  dbTable 'DimDate',
  tempDir 'wasbs://<CONTAINERNAME>@<BLOBNAME>.blob.core.windows.net/dbtblDimDate'
);

In [6]:
%sql
SELECT * FROM dbtblSalesByCategory limit 1

In [7]:
%sql
SELECT * FROM dbtblDimDate limit 10

In [8]:
%sql
SELECT * FROM dbtblFactInternetSales

In [9]:
%sql

SELECT A.CalendarYear, A.CalendarQuarter, A.MonthNumberOfYear, SUM(B.SalesAmount) AS TotalSalesAmount FROM dbtblDimDate AS A, dbtblFactInternetSales AS B WHERE A.DateKey = B.OrderDateKey GROUP BY A.CalendarYear, A.CalendarQuarter, A.MonthNumberOfYear ORDER BY A.CalendarYear, A.CalendarQuarter, A.MonthNumberOfYear

In [10]:
%sql
CREATE TABLE dbtblSalesByCalendarYear
USING PARQUET
AS SELECT A.CalendarYear, A.CalendarQuarter, A.MonthNumberOfYear, SUM(B.SalesAmount) AS TotalSalesAmount FROM dbtblDimDate AS A, dbtblFactInternetSales AS B WHERE A.DateKey = B.OrderDateKey GROUP BY A.CalendarYear, A.CalendarQuarter, A.MonthNumberOfYear ORDER BY A.CalendarYear, A.CalendarQuarter, A.MonthNumberOfYear

In [11]:
%sql
SELECT * FROM dbtblSalesByCalendarYear

In [12]:
%sql 
DROP TABLE IF EXISTS dwtblSalesByCalendarYear
CREATE TABLE dwtblSalesByCalendarYear
USING com.databricks.spark.sqldw
OPTIONS (
  url 'jdbc:sqlserver://<dbserver>.database.windows.net:1433;database=<dbName>;user=<USERID>@azlabf5;password=<PASSWORD>;encrypt=true;trustServerCertificate=false;hostNameInCertificate=*.database.windows.net;loginTimeout=30;',
  forwardSparkAzureStorageCredentials 'true',
  dbTable 'tblSalesByCalendarYear',
  tempDir 'wasbs://<CONTAINERNAME>@<BLOBNAME>.blob.core.windows.net/tblSalesByCalendarYear'
)
AS SELECT * FROM dbtblSalesByCalendarYear;

In [13]:
%sql
CREATE TABLE dbtblAVGSalesByCategory
USING PARQUET
AS SELECT ProductLine, AVG(SalesAmount) as AVGSalesAmount FROM dbtblSalesByCategory GROUP BY ProductLine